In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import json

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
#from sklearn.model_selection import train_test_split

from packaging import version
import pytorch_lightning as pl
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.optim as optimizers

from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")

In [3]:
all_new = pd.read_csv('data/feature_df_agg_svd16.csv')
BERT_svd256 = pd.read_csv("data/BERT_svd256.csv")
ALBERT_svd256 = pd.read_csv("data/ALBERT_svd256.csv")
ROBERTA_svd256 = pd.read_csv("data/ROBERTA_svd256.csv")
XLMROBERTA_svd256 = pd.read_csv("data/XLMROBERTA_svd256.csv")
tfidf_svd_raw_128 = pd.read_csv("data/tfidf_svd_raw_128_2gram.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [4]:
all_new = pd.merge(all_new, BERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ALBERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, XLMROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_128, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df, on="id", how="outer")

In [5]:
# min-max normalize

with open("data/base_feature_config_lgmb.json") as f:
    feature_config = json.load(f)
base_features = feature_config["selected_feature"]
base_features.remove("category2")

data = all_new[base_features]
dataf=((data-data.min())/(data.max()-data.min()))*6-3
all_new[base_features] = dataf

In [6]:
test_data = all_new[all_new["data_type"] == "test"]
train_data = all_new[all_new["data_type"] == "train"]

In [7]:
class CFDataset(Dataset):
    def __init__(self, csv_file, feature_cols, transform=None):
        self.csv_file = csv_file
        self.transform = transform
        self.feature_cols = feature_cols
        self.target_cols = ["state"]
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)
        
        target = self.csv_file[self.target_cols].iloc[idx]
        target = torch.tensor(target)
        
        if self.transform:
            features = self.transform(features)
            
        return features, target

In [8]:
class CFDataModule(pl.LightningDataModule):
    def __init__(self, train, validation, feature_cols, transform, batch_size, num_workers):
        super().__init__()
        self.csv_file = train
        self.transform = transform
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.validation = validation
        self.feature_cols = feature_cols
        

    def setup(self, stage=None):
        self.train_dataset = CFDataset(csv_file=self.csv_file, feature_cols=self.feature_cols, transform=self.transform)
        self.val_dataset = CFDataset(csv_file=self.validation, feature_cols=self.feature_cols, transform=self.transform)
            
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )

In [9]:
class CFModule(pl.LightningModule):
    def __init__(self, num_features, num_classes, trial):
        super(CFModule, self).__init__()
        self.layers = []
        self.dense_layers = []
        self.dropouts = []

        # We optimize the number of layers, hidden units in each layer and dropouts.
        n_layers = trial.suggest_int("n_layers", 2, 4)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)
        input_dim = num_features
        for i in range(n_layers):
            output_dim = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
            self.layers.append(nn.BatchNorm1d(input_dim))
            self.dropouts.append(nn.Dropout(dropout))
            self.dense_layers.append(nn.utils.weight_norm(nn.Linear(input_dim, output_dim)))
            input_dim = output_dim

        self.layers.append(nn.Linear(input_dim, num_classes))

        # Assigning the layers as class variables (PyTorch requirement).
        # Parameters of a layer are returned when calling model.parameters(),
        # only if the layer is a class variable. Thus, assigning as class
        # variable is necessary to make the layer parameters trainable.
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        # Assigning the dropouts as class variables (PyTorch requirement), for
        # the same reason as above.
        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
        for idx, dense in enumerate(self.dense_layers):
            setattr(self, "dense{}".format(idx), dense)
            
    
    def forward(self, x):
        for layer, dropout, dense_layer in zip(self.layers, self.dropouts, self.dense_layers):
            x = layer(x)
            x = dropout(x)
            x = F.relu(dense_layer(x))
        x = self.layers[-1](x)
        
        return torch.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("vall_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        #print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        return {"preds": pred}
    
    
    def test_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs], dim=0).squeeze()
        preds = np.array(preds.tolist())
        return {"preds": preds}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.01, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        #pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(input=pred, target=t)
        #return F.mse_loss(input=pred, target=t)
    
    
    def metric(self, pred, t):
        t = t.to('cpu')
        pred = np.array(pred.tolist())
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [10]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [11]:
def objective_configurer(MODEL_DIR, feature_cols, seed):
    def objective(trial):
        # data module config
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        for fold, (t_idx, v_idx) in enumerate(cv.split(train_data, train_data["category2"])):
            train_set = train_data.loc[t_idx]
            val_set = train_data.loc[v_idx]

        transform = None
        batch_size = 256*8
        num_workers = 4
        PERCENT_VALID_EXAMPLES = 1
        EPOCHS = 5

        # model config
        num_input = len(feature_cols)
        num_classes = 1

        cf = CFDataModule(train=train_set, validation=val_set, feature_cols=feature_cols,
                          transform=transform, batch_size=batch_size, num_workers=num_workers)
        model = CFModule(num_features=num_input, num_classes=num_classes, trial=trial)

        # Filenames for each trial must be made unique in order to access each checkpoint.
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            os.path.join(MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
        )

        # The default logger in PyTorch Lightning writes to event files to be consumed by
        # TensorBoard. We don't use any logger here as it requires us to implement several abstract
        # methods. Instead we setup a simple callback, that saves metrics from each validation step.
        metrics_callback = MetricsCallback()
        trainer = pl.Trainer(
            logger=False,
            limit_val_batches=PERCENT_VALID_EXAMPLES,
            checkpoint_callback=checkpoint_callback,
            max_epochs=EPOCHS,
            gpus=1 if torch.cuda.is_available() else None,
            callbacks=[metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
        )

        trainer.fit(model, cf)
        torch.cuda.empty_cache()

        return metrics_callback.metrics[-1]["val_loss"].item()
    return objective

In [12]:
def run_optuna_nn(feature_cols, seed):
    DIR = os.getcwd()
    MODEL_DIR = os.path.join(DIR, "result")
    
    pruner = optuna.pruners.NopPruner()

    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective_configurer(MODEL_DIR, feature_cols, seed), n_trials=60, timeout=int(1*60*60))

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    shutil.rmtree(MODEL_DIR)
    
    return trial.params

In [13]:
if __name__ == "__main__":
    
    with open("data/base_feature_config_lgmb.json") as f:
        feature_config = json.load(f)
    base_features = feature_config["selected_feature"]
    base_features.extend(list(mfw_df.columns.drop("id")))
    base_features.extend(list(miw_df.columns.drop("id")))
    
    CATEGORICAL_COLS = [
            "country",
            "category1",
            "category2",
            "country+category1",
            "country+category2",
            "category1+category2",
        ]
    
    additional_features = [
        (list(BERT_svd256.columns.drop("id")), "BERT"),
        (list(ALBERT_svd256.columns.drop("id")), "ALBERT"),
        (list(ROBERTA_svd256.columns.drop("id")), "ROBERTA"),
        (list(XLMROBERTA_svd256.columns.drop("id")), "XLMROBERTA"),
        (list(tfidf_svd_raw_128.columns.drop("id")), "tfidf"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(BERT_svd256.columns.drop("id")), "tfidf+BERT"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(ALBERT_svd256.columns.drop("id")), "tfidf+ALBERT"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(ROBERTA_svd256.columns.drop("id")), "tfidf+ROBERTA"),
        (list(tfidf_svd_raw_128.columns.drop("id")) + list(XLMROBERTA_svd256.columns.drop("id")), "tfidf+XLMROBERTA"),
    ]
    
    for n, additional_feature in enumerate(additional_features):
        additional_feature, feature_name = additional_feature
        print(f"============ {feature_name} optuna serching ============")
        feature_cols = base_features + additional_feature
        
        for categorical_col in set(CATEGORICAL_COLS):
            if categorical_col in feature_cols:
                feature_cols.remove(categorical_col)
        
        best_params = run_optuna_nn(feature_cols=feature_cols,
                                   seed=n*2022
                                   )
        
        
        config_dict = {
        "selected_feature": feature_cols,
        "best_params": best_params,
        "seed": n*2022
        }
        
        with open(f"data/optuna_nn_bce_{feature_name}_seed2022.json", 'w') as f:
            json.dump(config_dict, f)

[I 2021-02-02 19:12:33,205] A new study created in memory with name: no-name-7c1db921-7bf1-4468-837c-b305fbde8444


============ BERT optuna serching ============


/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 992   
1 | fc1    | BatchNorm1d | 50    
2 | fc2    | BatchNorm1d | 66    
3 | fc3    | Linear      | 8     
4 | drop0  | Dropout     | 0     
5 | drop1  | Dropout     | 0     
6 | drop2  | Dropout     | 0     
7 | dense0 | Linear      | 12.5 K
8 | dense1 | Linear      | 891   
9 | dense2 | Linear      | 245   
---------------------------------------
14.7 K    Trainable params
0         Non-trainable params
14.7 K    Total params


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


...

[I 2021-02-03 04:20:48,558] Trial 24 finished with value: 0.5259854197502136 and parameters: {'n_layers': 2, 'dropout': 0.2950641461751192, 'n_units_l0': 21, 'n_units_l1': 4}. Best is trial 15 with value: 0.516339898109436.
/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 1.2 K 
1 | fc1    | BatchNorm1d | 12    
2 | fc2    | Linear      | 14    
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 3.8 K 
6 | dense1 | Linear      | 104   
---------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 

/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


[I 2021-02-03 04:23:11,832] Trial 25 finished with value: 0.5251332521438599 and parameters: {'n_layers': 2, 'dropout': 0.4644247342773468, 'n_units_l0': 6, 'n_units_l1': 13}. Best is trial 15 with value: 0.516339898109436.



Number of finished trials: 26
Best trial:
  Value: 0.516339898109436
  Params: 
    n_layers: 2
    dropout: 0.3846782952403129
    n_units_l0: 9
    n_units_l1: 6


In [14]:
class CFModule(pl.LightningModule):
    def __init__(self, input_dim, hidden_sizes, out_dim, dropout):
        super(CFModule, self).__init__()
        self.layers = []
        self.dropouts = []
        self.dense_layers = []
        
        for hidden_size in hidden_sizes:
            self.layers.append(nn.BatchNorm1d(input_dim))
            self.dropouts.append(nn.Dropout(dropout))
            self.dense_layers.append(nn.utils.weight_norm(nn.Linear(input_dim, hidden_size)))
            input_dim = hidden_size
        self.layers.append(nn.Linear(input_dim, out_dim))
        
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
        for idx, dense in enumerate(self.dense_layers):
            setattr(self, "dense{}".format(idx), dense)

    
    def forward(self, x):
        for batch_norm, dropout, dense in zip(self.layers, self.dropouts, self.dense_layers):
            x = batch_norm(x)
            x = dropout(x)
            x = F.relu(dense(x))    
        x = self.layers[-1](x)
        
        return torch.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("val_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        return {"preds": pred}
    
    
    def test_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs], dim=0).squeeze()
        preds = np.array(preds.tolist())
        return {"preds": preds}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.01, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(input=pred, target=t)
        #return F.mse_loss(input=pred, target=t)
    
    
    def metric(self, pred, t):
        t = t.to('cpu')
        pred = np.array(pred.tolist())
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [15]:
def add_cols_kfold(n_folds, train, y_train, seed):
    cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    for fold, (t_idx, v_idx) in enumerate(cv.split(train, y_train)):
        train.loc[v_idx, 'kfold'] = int(fold)
        
    return train

In [16]:
def run_nn(optuna_config_dict, output_path):
    # trainer config
    max_epochs = 100
    
    
    # data module config
    #train_set
    transform = None
    batch_size = 256*8
    num_workers = 4
    
    # model config
    feature_cols = optuna_config_dict["selected_feature"]
    params = optuna_config_dict["best_params"]
    n_layers = params["n_layers"]
    dropout = params["dropout"]
    hidden_sizes = [0]*n_layers
    for n in range(n_layers):
        hidden_sizes[n] = params[f"n_units_l{n}"]
    input_dim = len(feature_cols)
    out_dim = 1
    
    # early stopping config
    patience = 4
    
    # kfold config
    n_folds = 5
    seed = optuna_config_dict["seed"]
    
    config_dict = {
        "max_epochs": max_epochs,
        "output_path": output_path,
        "batch_size": batch_size,
        "num_workers": num_workers,
        "optuna_config" : optuna_config_dict,
        "patience": patience,
        "n_folds": n_folds,
        "seed": seed 
    }
    
    # prepare data for with kfold
    train = add_cols_kfold(n_folds=n_folds,
                            train=train_data,
                           y_train=train_data["category2"],
                            seed=seed)
    
    test_prediction = []
    oof_train = np.zeros((len(train_data),))
    for fold in train.kfold.unique():
        train_fold = train[train["kfold"] != fold]
        val_fold = train[train["kfold"] == fold]
        val_idx = val_fold.index
    
        cf = CFDataModule(train=train_fold, validation=val_fold, feature_cols=feature_cols,
                          transform=transform, batch_size=batch_size, num_workers=num_workers)
        model = CFModule(input_dim=input_dim, hidden_sizes=hidden_sizes, out_dim=out_dim, dropout=dropout)

        early_stopping = EarlyStopping('val_loss', patience=patience, verbose=True)
        trainer = Trainer(
            max_epochs=max_epochs,
            weights_save_path=output_path,
            gpus = 1 if torch.cuda.is_available() else None,
            callbacks=[early_stopping],
            #accumulate_grad_batches=1
            # use_amp=False,
            )
        
        # training
        trainer.fit(model, cf)
        
        # oof prediction
        oof_dataset = CFDataset(val_fold, feature_cols)
        oof_dataloader = DataLoader(oof_dataset,
                                    batch_size=batch_size,
                                    drop_last=False,
                                    shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True)
        oof_results = trainer.test(test_dataloaders=oof_dataloader, verbose=False)
        oof_train[val_idx] = oof_results[0]["preds"]
        
        # test prediction
        test_dataset = CFDataset(test_data, feature_cols)
        test_dataloader = DataLoader(test_dataset,
                                    batch_size=batch_size,
                                    drop_last=False,
                                     shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True)
        test_results = trainer.test(test_dataloaders=test_dataloader, verbose=False)
        test_preds = test_results[0]["preds"]
        test_prediction.append(test_preds)
        torch.cuda.empty_cache()
        
    return oof_train, sum(test_prediction)/len(test_prediction), config_dict

In [17]:
if __name__ == "__main__":
    
    feature_names = [
        "ALBERT",
        "BERT",
        "ROBERTA",
        "XLMROBERTA",
        "tfidf",
        "tfidf+ALBERT",
        "tfidf+BERT",
        "tfidf+ROBERTA",
        "tfidf+XLMROBERTA",
    ]
    
    for feature_name in feature_names:
        with open(f"data/optuna_nn_bce_{feature_name}_seed2022.json") as f:
            optuna_config_dict = json.load(f)
        
        output_path = f"./nn_logs/{feature_name}"
        oof, preds, config_dict = run_nn(optuna_config_dict, output_path)
        
        config_dict["out_of_fold"] = list(oof)
        config_dict["prediction"] = list(preds)
        with open(f"data/nn_bce_{feature_name}_seed2022.json", 'w') as f:
            json.dump(config_dict, f)

/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/teppei/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/teppei/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

val_loss: 0.6948710680007935, val_acc: 0.6735751295336787


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


val_loss: 0.6603981256484985, val_acc: 0.11330935251798561


...

val_loss: 0.4300693869590759, val_acc: 0.7993981945837514


val_loss: 0.4329279065132141, val_acc: 0.7961847389558232


val_loss: 0.4318620562553406, val_acc: 0.7978142076502731


val_loss: 0.4295453429222107, val_acc: 0.7996051332675223



/home/teppei/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The testing_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
